##O MODELO ESTA BUGADO, ENTRETANTO COMPLETO

In [0]:
import pandas as pd;
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle


from sklearn.metrics import accuracy_score
from scipy.stats import expon, binom, t, norm, probplot, scoreatpercentile, percentileofscore
import seaborn as sns # evite fazer pairplot! Estoura a memória
import statsmodels.api as sm 
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# %matplotlib inline
# df1.to_excel(r'C:\Users\Antonio Fonseca\Desktop\BLOX\ementas.xlsx')
# n esquecer do tensorflow

In [2]:
# treino = pd.read_csv('treino.csv')
url = 'https://raw.githubusercontent.com/Antonioarf/blox/master/treino.csv'
df1 = pd.read_csv(url)
treino = df1
treino.shape
treino.head(3)

,Unnamed: 0,materias,ÁREA DO CONHECIMENTO_x,obj,p_r,programa
0,0,Acolhimento em Emergências,programas genericos,1. Compreender os desafios ético-sociais envol...,O Objetivo geral é que o aluno tenha noção do ...,Atividades Teóricas: \n1.Macro e Micro regulaç...
1,1,Apoio à Escolha da Especialidade Médica,programas genericos,Compreender os principais fatores que influenc...,Estudo de fatores ligados ao estudante e à nat...,Fatores que influenciam a escolha da especiali...
2,2,"Atividades Acadêmicas Científicas, de Extensão...",programas genericos,"Estimular a pró-atividade dos alunos, incentiv...",São computadas as participações em: atividades...,1. Atividades de extensão: cursos de difusão c...


In [3]:
def deletar_caracteres(coluna):
    coluna = coluna.replace("Ciências Naturais","1").replace("Matemática e Computação","2").replace("Engenharia, Produção e Construção","3").replace("Saúde e Bem-Estar Social","4").replace("Humanidades e Linguagens","4")\
    .replace('Ciências Sociais, Jornalismo e Informação', "5").replace("programas genericos","6").replace("Negócios e Administração","7").replace("Agricultura e Veterinária", "8").replace("Serviços", "9").replace("Educação", "10")\
    .replace("Programas Interdisciplinares", "11").replace("Psicologia e Desenvolvimento", "12").replace("Direito", "13")
    return coluna
treino =  treino.dropna()
treino.shape

(3265, 6)

In [0]:
y = deletar_caracteres(treino["ÁREA DO CONHECIMENTO_x"])
treino = treino.drop(["ÁREA DO CONHECIMENTO_x", "Unnamed: 0",'p_r'], axis=1)
# y = y.to_numpy()


In [5]:
treino["tudo"] = treino["obj"] + treino["programa"]
x = treino["tudo"]
# X_train, X_test, y_train, y_test = train_test_split(treino,y, test_size=0.3,random_state=42)
y.value_counts()

4     700
1     492
2     445
3     425
5     317
6     178
7     169
8     149
9     103
10     94
11     83
12     77
13     33
Name: ÁREA DO CONHECIMENTO_x, dtype: int64

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(x)
# vectorizer.vocabulary_

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
                lowercase=False, max_df=1.0, max_features=None, min_df=0,
                ngram_range=(1, 1), preprocessor=None, stop_words=None,
                strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, vocabulary=None)

In [7]:
vectorizer.transform(x).toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [0]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=1000)

In [9]:
vectorizer = CountVectorizer()
vectorizer.fit(x_train)

X_train = vectorizer.transform(x_train)
X_test  = vectorizer.transform(x_test)
y_train

1941    1
3490    4
2999    3
1244    7
1390    5
       ..
1672    1
2873    3
2559    2
732     4
1784    1
Name: ÁREA DO CONHECIMENTO_x, Length: 2285, dtype: object

In [10]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy:", score)

Accuracy: 0.7306122448979592


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [11]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]  # Number of features

model = Sequential()
model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))


Using TensorFlow backend.


In [12]:
model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 10)                245880    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 11        
Total params: 245,891
Trainable params: 245,891
Non-trainable params: 0
_________________________________________________________________


In [13]:
history = model.fit(X_train, y_train, epochs=100, verbose=False, validation_data=(X_test, y_test), batch_size=10)

In [14]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(X_test, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Accuracy: 0.1414
Testing Accuracy:  0.1724
